In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Optimizer
import torch.backends.cudnn as cudnn
import tqdm

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader,Subset
import torchvision.models as models
import torch.nn.functional as F
from models import *

import os
import copy
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from util import *
import time
from my_utils import *


random_seed = 0
np.random.seed(random_seed)
random.seed(random_seed)
torch.manual_seed(random_seed)

In [21]:
#The argumention use for surrogate model training stage
transform_surrogate_train = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomCrop(32, padding=4),  
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])



#The argumention use for all testing set
transform_test = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
dataset_path = './data/'
#外面的数据集加载进来
outter_trainset = My_PoisonFolder(1, root=dataset_path + 'tiny-imagenet-200/train/', \
                                  transform=transform_surrogate_train)

frog_trainset = My_PoisonFolder(0, root=dataset_path + 'tiny-imagenet-200/frog/', \
                                  transform=transform_surrogate_train)

total_set = concoct_dataset(outter_trainset, frog_trainset) 

In [22]:
train_batch_size = 350
outter_loader = torch.utils.data.DataLoader(total_set, batch_size=train_batch_size, shuffle=True, num_workers=16)

In [34]:
other_loader = torch.utils.data.DataLoader(outter_trainset, batch_size=train_batch_size, shuffle=True, num_workers=16)

frog_loader = torch.utils.data.DataLoader(frog_trainset, batch_size=train_batch_size, shuffle=True, num_workers=16)


In [29]:
decode_model = ResNet18_201()
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    # 将batchsize 30 分配到N个GPU上运行
    decode_model = nn.DataParallel(decode_model)
device = torch.device("cuda:0" if True else "cpu")
decode_model = decode_model.to(device)

Let's use 2 GPUs!


In [30]:
criterion = torch.nn.CrossEntropyLoss()
# outer_opt = torch.optim.RAdam(params=base_model.parameters(), lr=generating_lr_outer)
optimizer = torch.optim.SGD(params=decode_model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [31]:
time_list = []

for epoch in range(0, 200):
    since = time.time()
    decode_model.train()
    loss_list = []
    for images, labels in outter_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = decode_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        loss_list.append(float(loss.data))
        optimizer.step()
        
    scheduler.step()
    
    ave_loss = np.average(np.array(loss_list))

    time_elapsed = time.time() - since
    print('Epoch:%d, Loss: %.03f, time:%0.3f s' % (epoch, ave_loss, time_elapsed))
    since = time.time()
    #record time
    time_list.append(time_elapsed)
#Save the surrogate model
save_path = './checkpoint_my/decode_pretrain_binary' + str(200) +'.pth'
torch.save(decode_model.state_dict(),save_path)

Epoch:0, Loss: 0.137, time:24.586 s
Epoch:1, Loss: 0.057, time:25.637 s
Epoch:2, Loss: 0.063, time:24.446 s
Epoch:3, Loss: 0.053, time:24.454 s
Epoch:4, Loss: 0.052, time:24.892 s
Epoch:5, Loss: 0.052, time:24.960 s
Epoch:6, Loss: 0.051, time:24.136 s
Epoch:7, Loss: 0.050, time:25.217 s
Epoch:8, Loss: 0.049, time:24.925 s
Epoch:9, Loss: 0.049, time:24.425 s
Epoch:10, Loss: 0.048, time:24.233 s
Epoch:11, Loss: 0.047, time:24.500 s
Epoch:12, Loss: 0.046, time:24.990 s
Epoch:13, Loss: 0.045, time:24.307 s
Epoch:14, Loss: 0.044, time:24.588 s
Epoch:15, Loss: 0.044, time:25.082 s
Epoch:16, Loss: 0.044, time:24.492 s
Epoch:17, Loss: 0.042, time:25.239 s
Epoch:18, Loss: 0.042, time:24.265 s
Epoch:19, Loss: 0.041, time:24.469 s
Epoch:20, Loss: 0.040, time:24.639 s
Epoch:21, Loss: 0.040, time:24.153 s
Epoch:22, Loss: 0.039, time:24.662 s
Epoch:23, Loss: 0.039, time:24.077 s
Epoch:24, Loss: 0.038, time:24.417 s
Epoch:25, Loss: 0.037, time:23.908 s
Epoch:26, Loss: 0.037, time:23.979 s
Epoch:27, L

In [36]:
d2l.evaluate_accuracy_gpu(decode_model, other_loader)

1.0